# Análise de Experimento - Teste A/B

**Autor:** Anderson Ramos
**Data:** 16/10/2025

## Objetivo

O objetivo desta análise é avaliar os resultados de um teste A/B para determinar se uma nova página de destino (grupo "treatment") gera uma taxa de conversão maior do que a página antiga (grupo "control"). A análise será conduzida com rigor estatístico para garantir que qualquer decisão de negócio seja baseada em evidências sólidas.

In [19]:
# 1. Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest

# 2. Carregamento do dataset
try:
    # Use o nome exato do arquivo que você baixou
    df = pd.read_csv('ab_data.csv') 
    print("Dataset carregado com sucesso!")
    print(f"O dataset contém {df.shape[0]} linhas e {df.shape[1]} colunas.")
except FileNotFoundError:
    print("ERRO: Arquivo 'ab_data.csv' não encontrado. Verifique se o nome e o caminho estão corretos.")

# 3. Visualização inicial dos dados
df.head()

Dataset carregado com sucesso!
O dataset contém 294480 linhas e 5 colunas.


,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [20]:
# Verificando informações básicas do DataFrame
df.info()

# Verificando o número de usuários únicos
print(f"\nNúmero de usuários únicos no dataset: {df.user_id.nunique()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  int64 
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB

Número de usuários únicos no dataset: 290585


In [21]:
# Checando se usuários do grupo 'treatment' realmente viram a 'new_page'
# E se usuários do 'control' realmente viram a 'old_page'

# Contagem de inconsistências
inconsistencias = df.query("(group == 'treatment' and landing_page != 'new_page') or (group == 'control' and landing_page != 'old_page')")

print(f"Número de linhas com inconsistências: {len(inconsistencias)}")

Número de linhas com inconsistências: 3893


In [22]:
# O índice das linhas inconsistentes
indices_inconsistentes = inconsistencias.index

# Removemos as linhas inconsistentes
df_limpo = df.drop(indices_inconsistentes)

# Verificamos e removemos qualquer usuário duplicado, mantendo apenas a primeira ocorrência
df_limpo = df_limpo.drop_duplicates(subset='user_id', keep='first')

print("Inconsistências e duplicatas removidas.")
print(f"Novo tamanho do dataset: {df_limpo.shape[0]} linhas.")

Inconsistências e duplicatas removidas.
Novo tamanho do dataset: 290585 linhas.


## 2. Formulação da Hipótese Estatística

Para tomar uma decisão baseada em dados, precisamos formular duas hipóteses opostas.

*   **Hipótese Nula (H₀):** A nova página **não tem efeito ou tem um efeito negativo** na taxa de conversão. Matematicamente, a taxa de conversão da nova página (p_nova) é menor ou igual à da página antiga (p_antiga).
    
    `H₀: p_nova ≤ p_antiga`

*   **Hipótese Alternativa (H₁):** A nova página **tem um efeito positivo**, ou seja, aumenta a taxa de conversão.
    
    `H₁: p_nova > p_antiga`

Nosso objetivo é encontrar evidências estatísticas para **rejeitar a hipótese nula** em favor da hipótese alternativa.

*   **Nível de Significância (α):** Adotaremos um nível de significância de **5% (α = 0.05)**. Isso significa que estamos dispostos a aceitar uma probabilidade de 5% de rejeitar a hipótese nula quando ela é, na verdade, verdadeira (erro do Tipo I).

In [23]:
# Calculando a taxa de conversão geral
print(f"Taxa de conversão geral (dataset limpo): {df_limpo.converted.mean():.4f}")

# Separando os dados por grupo (controle e tratamento)
grupo_controle = df_limpo.query("group == 'control'")
grupo_tratamento = df_limpo.query("group == 'treatment'")

# Calculando a taxa de conversão para cada grupo
taxa_controle = grupo_controle.converted.mean()
taxa_tratamento = grupo_tratamento.converted.mean()

print(f"Taxa de Conversão do Grupo de Controle: {taxa_controle:.4f}")
print(f"Taxa de Conversão do Grupo de Tratamento: {taxa_tratamento:.4f}")

# Calculando o número de conversões e usuários em cada grupo (necessário para o teste)
conversoes_controle = grupo_controle.converted.sum()
usuarios_controle = len(grupo_controle)

conversoes_tratamento = grupo_tratamento.converted.sum()
usuarios_tratamento = len(grupo_tratamento)

Taxa de conversão geral (dataset limpo): 0.1196
Taxa de Conversão do Grupo de Controle: 0.1204
Taxa de Conversão do Grupo de Tratamento: 0.1188


In [24]:
# Preparando os dados para o teste Z
# Contagem de conversões [tratamento, controle]
count = np.array([conversoes_tratamento, conversoes_controle])

# Número total de observações [tratamento, controle]
nobs = np.array([usuarios_tratamento, usuarios_controle])

# Executando o teste Z
# O argumento 'alternative='larger'' testa se a proporção do primeiro grupo (tratamento)
# é maior que a do segundo (controle), o que corresponde exatamente à nossa hipótese H₁.
z_stat, p_valor = proportions_ztest(count, nobs, alternative='larger')

print(f"Estatística Z: {z_stat:.4f}")
print(f"P-valor: {p_valor:.4f}")

Estatística Z: -1.3116
P-valor: 0.9052


## 3. Análise dos Resultados e Conclusão

Após a realização do teste Z para duas proporções, obtivemos os seguintes resultados:

*   **Estatística Z:** [-1.3116]
*   **P-valor:** [0.9052]

### Interpretação Estatística

O nível de significância (α) que definimos para este teste foi de 0.05. Comparamos o p-valor obtido com este limiar:

**`P-valor ([0.9052])` vs `α (0.05)`**

*   **(Cenário 1: Se o seu p-valor for > 0.05)**
    Como o p-valor é **maior** que o nosso nível de significância (p > 0.05), **falhamos em rejeitar a Hipótese Nula (H₀)**. Isso significa que não encontramos evidências estatísticas suficientes para afirmar que a nova página tem uma taxa de conversão superior à da página antiga. A pequena diferença que observamos nas taxas de conversão pode ser meramente fruto do acaso.



### Recomendação de Negócio

**Baseado nos resultados do teste A/B, não há evidências estatísticas para justificar a implementação da nova página para todos os usuários.** Embora a taxa de conversão do grupo de tratamento tenha sido ligeiramente diferente da do grupo de controle, essa diferença não foi estatisticamente significante.

**Recomenda-se manter a página antiga (versão de controle) como padrão.** Investir recursos para lançar a nova página em larga escala não traria um retorno positivo garantido e poderia ser um desperdício de tempo e dinheiro. Sugere-se que o time de produto investigue novas hipóteses para melhorar a experiência do usuário.